In [ ]:
pip install torch transformers langchain gradio sacremoses

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import gradio as gr

In [ ]:
from transformers import BioGptForCausalLM, BioGptTokenizer,pipeline
import torch
from peft import PeftModel

base_model = BioGptForCausalLM.from_pretrained(
    "microsoft/biogpt",  # Original base model
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, "/kaggle/input/notebook36cd12f0a1/biogpt-qlora-medical")
model = model.merge_and_unload() 


In [ ]:
from datasets import load_from_disk

val_dataset = load_from_disk("/kaggle/input/notebook36cd12f0a1/val_dataset")


In [ ]:
pip install -U langchain langchain-core langchain-community langchain-huggingface


In [ ]:
from langchain_huggingface import HuggingFacePipeline

In [ ]:
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt") 
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.6,
    top_p=0.6,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id,
    prefix="",
)

from langchain_core.prompts.prompt import PromptTemplate

template = """
History:{history}
###Patient: {input}
###Doctor:"""





In [ ]:
# Initialize Langchain memory and conversation chain
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
memory = ConversationBufferMemory(k=2, return_messages=True,ai_prefix="Doctor",human_prefix="Patient")
llm = HuggingFacePipeline(pipeline=pipeline)

conversation = ConversationChain(llm=llm, memory=memory,prompt=PROMPT)

In [ ]:
from langchain.schema import HumanMessage, AIMessage

def extract_latest_doctor_response(text):
    last_index = text.rfind("Doctor:")
    if last_index != -1:
        return text[last_index + len("Doctor:"):].strip()
    else:
        return text.strip()
        
def format_history(messages):
    last_two_messages = messages[-2:]

    history_lines = []
    for msg in last_two_messages:
        if isinstance(msg, HumanMessage):
            history_lines.append(f"Patient: {msg.content.strip()}")
        elif isinstance(msg, AIMessage):
            history_lines.append(f"Doctor: {msg.content.strip()}")
    
    return "\n".join(history_lines)


def predict(input, history=None):
    formatted_history = format_history(memory.chat_memory.messages)

    prompt = PROMPT.format(history=formatted_history, input=input)

    raw_response = llm(prompt)

    cleaned_response = extract_latest_doctor_response(raw_response)

    memory.chat_memory.add_user_message(input)
    memory.chat_memory.add_ai_message(cleaned_response)

    return raw_response


# Create the Gradio UI
iface = gr.ChatInterface(
    fn=predict,
    title="BioGPT Doctor Chatbot",
    description="Ask BioGPT medical-related questions.",
    examples=[
        ["My knees are experiencing alot of pain."],
        ["I have a persistent cough and fever."],
        ["What are the common symptoms of diabetes?"],
    ],
    theme="soft",
)

iface.launch(share=True)